In [15]:
import pandas as pd
import numpy as np
import time

from selenium import webdriver

# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.by import By

# 크롬드라이버 옵션 (창 안보이게 숨기기)
driver_option = webdriver.ChromeOptions()
# driver_option.add_argument('headless')
driver = webdriver.Chrome(options=driver_option) 
driver.implicitly_wait(3)

In [2]:
data_path = './dataset_gachon_userinformation.csv'
df = pd.read_csv(data_path)
df

,userName,problemId,solved
0,rejo,1000,1
1,rejo,1001,1
2,rejo,1002,1
3,rejo,1003,1
4,rejo,1008,1
...,...,...,...
104522,ghdalsdud000,2557,1
104523,ghdalsdud000,1000,0
104524,ghdalsdud000,1001,0
104525,ghdalsdud000,1000,0


In [22]:
# 유저와 문제의 고유한 값을 추출
users = df['userName'].unique()
problems = df['problemId'].unique()

# 유저-문제 행렬 초기화
N_USER = len(users)
N_PROBLEM = 35000
user_item_matrix = np.zeros((N_USER, N_PROBLEM))

# 유저와 문제 ID를 위한 매핑 생성
user_to_idx = {user: idx for idx, user in enumerate(users)}

# 데이터를 바탕으로 행렬 채우기
for _, row in df.iterrows():
    user_idx = user_to_idx[row['userName']]
    problem_idx = row['problemId']
    user_item_matrix[user_idx, problem_idx] = row['solved']

# 결과 출력
user_item_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [23]:
user_item_matrix.shape  # 만들어진 user item interaction matrix의 크기

(663, 35000)

In [24]:
max(user_to_idx.values())  # 새로운 유저가 들어왓을 때 할당될 번호 - 1

662

In [16]:
user_name = 'rbrb0014'

interaction = []

driver.get(f'https://www.acmicpc.net/user/{user_name}')  # 백준 유저 정보 페이지 접속
time.sleep(3)

# tier_info = driver.find_elements(by=By.CSS_SELECTOR, value='.solvedac-tier')[0]
# tier_svg = tier_info.get_attribute('src').split("/")[-1]
# tier = int(tier_svg.split(".")[0])

# -- 유저가 해결한 문제
solved_problems = driver.find_elements(by=By.XPATH, value="//div[contains(@class, 'panel') and contains(@class, 'panel-default') and .//h3[contains(text(), '맞은 문제')]]//a")
for solved in solved_problems:
    interaction.append([user_name, solved.text, 1])

interaction

[['rbrb0014', '1000', 1],
 ['rbrb0014', '1001', 1],
 ['rbrb0014', '1003', 1],
 ['rbrb0014', '1008', 1],
 ['rbrb0014', '1049', 1],
 ['rbrb0014', '1065', 1],
 ['rbrb0014', '1110', 1],
 ['rbrb0014', '1149', 1],
 ['rbrb0014', '1152', 1],
 ['rbrb0014', '1157', 1],
 ['rbrb0014', '1238', 1],
 ['rbrb0014', '1260', 1],
 ['rbrb0014', '1330', 1],
 ['rbrb0014', '1389', 1],
 ['rbrb0014', '1463', 1],
 ['rbrb0014', '1546', 1],
 ['rbrb0014', '1613', 1],
 ['rbrb0014', '1717', 1],
 ['rbrb0014', '1753', 1],
 ['rbrb0014', '1890', 1],
 ['rbrb0014', '1904', 1],
 ['rbrb0014', '1912', 1],
 ['rbrb0014', '1920', 1],
 ['rbrb0014', '1932', 1],
 ['rbrb0014', '1946', 1],
 ['rbrb0014', '1956', 1],
 ['rbrb0014', '1991', 1],
 ['rbrb0014', '1992', 1],
 ['rbrb0014', '2156', 1],
 ['rbrb0014', '2164', 1],
 ['rbrb0014', '2167', 1],
 ['rbrb0014', '2193', 1],
 ['rbrb0014', '2292', 1],
 ['rbrb0014', '2293', 1],
 ['rbrb0014', '2438', 1],
 ['rbrb0014', '2439', 1],
 ['rbrb0014', '2440', 1],
 ['rbrb0014', '2441', 1],
 ['rbrb0014'

In [25]:
new_user_interaction = np.zeros(N_PROBLEM)
for row in interaction:
    new_user_interaction[int(row[1])] = 1

new_user_interaction

array([0., 0., 0., ..., 0., 0., 0.])

In [27]:
from sklearn.metrics.pairwise import cosine_similarity
# 새로운 사용자 데이터 추가
# 예를 들어, 새 사용자가 문제 101과 104를 풀었다고 가정합니다.
new_user_data = new_user_interaction  # 이 배열은 새 사용자가 푼 문제들을 나타냅니다.
user_item_matrix = np.vstack([user_item_matrix, new_user_data])

# 코사인 유사도 계산
cosine_sim = cosine_similarity(user_item_matrix)

# 새 사용자와 기존 사용자들 간의 유사도
new_user_similarity = cosine_sim[-1][:-1]

# 가장 유사한 사용자 찾기
top_users = np.argsort(new_user_similarity)[::-1][:5]  # 상위 3명의 유사한 사용자

# 새 사용자에 대한 추천 생성
# 새 사용자가 풀지 않은 문제를 찾고, 유사한 사용자들이 푼 문제들을 기반으로 추천
recommended_problems = set()
for user in top_users:
    solved_by_user = np.where(user_item_matrix[user] == 1)[0]
    for problem in solved_by_user:
        if new_user_data[problem] == 0:  # 새 사용자가 풀지 않은 문제
            recommended_problems.add(problem)
# 여기는 유사도가 높은 사람 5명 중 한 사람이라도 풀었으면 대입하게 됨

# 결과 출력
print("추천 문제:", recommended_problems)

추천 문제: {1316, 2558, 1193, 10250, 2941, 1712, 9012, 2869, 5622, 10807, 10872, 3036, 5597, 2750}


In [ ]:
import time
import random
import pickle
import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from sklearn.metrics.pairwise import cosine_similarity


PICKLE_SAVE_PATH = "./asset/user_based_collaborative_filtering.pickle"
DATA_PATH = './data/dataset_gachon_userinformation.csv'
N_PROBLEM = 35000

def train():

    # -- 가천대학교 학생의 문제풀이 이력이 담긴 데이터셋
    df = pd.read_csv(DATA_PATH)

    # 유저와 문제의 고유한 값을 추출
    users = df['userName'].unique()

    # 유저-문제 행렬 초기화
    N_USER = len(users)

    user_item_matrix = np.zeros((N_USER, N_PROBLEM))

    # 유저와 문제 ID를 위한 매핑 생성
    user_to_idx = {user: idx for idx, user in enumerate(users)}

    # 데이터를 바탕으로 행렬 채우기
    for _, row in df.iterrows():
        user_idx = user_to_idx[row['userName']]
        problem_idx = row['problemId']
        user_item_matrix[user_idx, problem_idx] = row['solved']

    with open(PICKLE_SAVE_PATH, 'wb') as file:
        pickle.dump(user_item_matrix, file)


def predict(user_id: str, threshold: int):

    with open(PICKLE_SAVE_PATH, 'rb') as file:
        user_item_matrix = pickle.load(file)

    interaction = []

    # -- 크롬드라이버 옵션 (창 안보이게 숨기기)
    df = pd.read_csv("./data/dataset_gachon_user_information.csv")
    df.groupby(['u'])

    try:
        driver_option = webdriver.ChromeOptions()
        driver_option.add_argument("headless")

        driver = webdriver.Chrome(options=driver_option) 
        driver.get(f'https://www.acmicpc.net/user/{user_id}')  # 백준 유저 정보 페이지 접속
        print(f"[DEBUG] https://www.acmicpc.net/user/{user_id} 에 접속해서 유저가 풀이한 목록 확인 중...")
        time.sleep(3)

        # -- 유저가 해결한 문제
        solved_problems = driver.find_elements(by=By.XPATH, value="//div[contains(@class, 'panel') and contains(@class, 'panel-default') and .//h3[contains(text(), '맞은 문제')]]//a")
        for solved in solved_problems:
            interaction.append([user_id, solved.text, 1])
    except:
        print("[DEBUG] Error occurred at chrome driver. please check the internet connection ㅠㅠ")
    finally:
        driver.close()
        print("[DEBUG] 크롬 드라이버 정상적으로 종료 완료")
    
    
    new_user_interaction = np.zeros(N_PROBLEM)
    for row in interaction:
        new_user_interaction[int(row[1])] = 1
    
    # 새로운 사용자 데이터 추가
    # 예를 들어, 새 사용자가 문제 101과 104를 풀었다고 가정합니다.
    new_user_data = new_user_interaction  # 이 배열은 새 사용자가 푼 문제들을 나타냅니다.
    user_item_matrix = np.vstack([user_item_matrix, new_user_data])

    # 코사인 유사도 계산
    cosine_sim = cosine_similarity(user_item_matrix)

    # 새 사용자와 기존 사용자들 간의 유사도
    new_user_similarity = cosine_sim[-1][:-1]

    # 가장 유사한 사용자 찾기
    top_users = np.argsort(new_user_similarity)[::-1][:threshold]  # 상위 5명의 유사한 사용자

    # 새 사용자에 대한 추천 생성
    # 새 사용자가 풀지 않은 문제를 찾고, 유사한 사용자들이 푼 문제들을 기반으로 추천
    recommended_problems = set()
    for user in top_users:
        solved_by_user = np.where(user_item_matrix[user] == 1)[0]
        for problem in solved_by_user:
            if new_user_data[problem] == 0:  # 새 사용자가 풀지 않은 문제
                recommended_problems.add(problem)
                

    # idea 1. 왜 추천되었는지 설명해주기
    # - 어떤 학생과 몇 %의 유사도때문에 이것이 추천되었는지 알려주기

    # 결과 출력
    recommended_problems = list(recommended_problems)
    random.shuffle(recommended_problems)
    return recommended_problems[:5]

In [14]:
import pandas as pd

df = pd.read_csv("./data/dataset_gachon_userinformation.csv")
solved_df = df[df['solved'] == 1]
solved_df.groupby(['userName'])['problemId'].apply(list)['0917jong']

[1000,
 1001,
 1002,
 1003,
 1004,
 1005,
 1006,
 1007,
 1008,
 1009,
 1010,
 1011,
 1012,
 1013,
 1014,
 1015,
 1016,
 1017,
 1018,
 1019,
 1021,
 1022,
 1024,
 1025,
 1026,
 1028,
 1029,
 1031,
 1032,
 1034,
 1037,
 1038,
 1039,
 1041,
 1043,
 1049,
 1051,
 1052,
 1057,
 1058,
 1059,
 1063,
 1064,
 1065,
 1067,
 1068,
 1072,
 1074,
 1075,
 1076,
 1080,
 1082,
 1085,
 1086,
 1089,
 1091,
 1092,
 1094,
 1099,
 1100,
 1102,
 1103,
 1105,
 1106,
 1107,
 1110,
 1120,
 1124,
 1126,
 1138,
 1145,
 1149,
 1152,
 1157,
 1158,
 1159,
 1162,
 1166,
 1167,
 1181,
 1182,
 1188,
 1189,
 1193,
 1194,
 1197,
 1201,
 1202,
 1205,
 1208,
 1210,
 1212,
 1213,
 1214,
 1222,
 1225,
 1229,
 1231,
 1233,
 1235,
 1236,
 1238,
 1240,
 1244,
 1246,
 1251,
 1252,
 1253,
 1254,
 1256,
 1258,
 1259,
 1260,
 1261,
 1263,
 1264,
 1267,
 1269,
 1271,
 1275,
 1280,
 1283,
 1285,
 1292,
 1297,
 1298,
 1300,
 1302,
 1303,
 1305,
 1306,
 1309,
 1312,
 1316,
 1321,
 1325,
 1328,
 1330,
 1334,
 1339,
 1343,
 1344,
 1351,

In [33]:
from collections import Counter
df = pd.read_csv("./data/dataset_problem_information.csv")
diffs = df[df['solved'] >= 400]
diffs

,problemId,problemTitle,solved,submissionCount,correctRatio,difficulty
0,1000,"""A+B""",268961,974437,39.624,1.0
1,1001,"""A-B""",229710,397369,69.902,1.0
2,1002,"""터렛""",35027,204710,22.429,8.0
3,1003,"""피보나치 함수""",49106,205120,32.818,8.0
4,1004,"""어린 왕자""",14541,38627,45.830,8.0
...,...,...,...,...,...,...
28776,30454,"""얼룩말을 찾아라!""",566,987,64.983,3.0
28777,30455,"""이제는 더 이상 물러날 곳이 없다""",512,596,92.086,5.0
28790,30468,"""호반우가 학교에 지각한 이유 1""",420,620,73.684,2.0
28823,30501,"""관공... 어찌하여 목만 오셨소...""",437,556,85.019,0.0
